In [1]:
#Importando bibliotecas nescessárias 
import pandas as pd 
import json
import psycopg2
from sqlalchemy import create_engine
from test_functions import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

**Lendo dados**

In [2]:
config_file = open('../config/data_config.json')
config_data = json.loads(config_file.read())
config_file.close()

In [3]:
test_file = open('../config/test_config.json')
config_test = json.loads(test_file.read())
test_file.close()

**Conectando**

In [4]:
connect_data = config_data['connection']
textEngine = f"{connect_data['driver']}://{connect_data['user']}:{connect_data['password']}@{connect_data['host']}:{connect_data['port']}/{connect_data['database']}"

**Buscando dados**

In [5]:
df = pd.read_sql_table(config_data['tables'][0],textEngine)

**Fazendo verificações**

In [6]:
out_df = {}

In [7]:
def calc_percent_cat(df):
    count = 0
    for type_table in df.dtypes:
        if type_table == "object" or str(type_table) == "category":
            count += 1
    print(count)
    return count/len(df.columns) 

out_df['columns'] = df.columns
out_df['types'] = dict(df.dtypes)
out_df['null_percent'] = dict(df.isnull().sum()/len(df))
out_df['n_unique'] = dict(df.nunique())
out_df['category_percent'] = dict(df.nunique())

In [8]:
out_df

{'columns': Index(['index', 'ProductID', 'Date', 'Zip', 'Units', 'Revenue'], dtype='object'),
 'types': {'index': dtype('int64'),
  'ProductID': dtype('int64'),
  'Date': dtype('O'),
  'Zip': dtype('int64'),
  'Units': dtype('int64'),
  'Revenue': dtype('O')},
 'null_percent': {'index': 0.0,
  'ProductID': 0.0,
  'Date': 0.0,
  'Zip': 0.0,
  'Units': 0.0,
  'Revenue': 3.252027361447771e-05},
 'n_unique': {'index': 1260752,
  'ProductID': 1893,
  'Date': 5149,
  'Zip': 32595,
  'Units': 36,
  'Revenue': 3774},
 'category_percent': {'index': 1260752,
  'ProductID': 1893,
  'Date': 5149,
  'Zip': 32595,
  'Units': 36,
  'Revenue': 3774}}

**Todos as funções**

In [9]:
list_functions = [test_columns,test_types,test_null_percent,test_n_uniques,test_category_percent]
out_data = {}
for function in list_functions:
    name_test = function.__name__
    retorno = function(df,config_test['salesfacts'])
    out_data[name_test] = retorno

In [10]:
config_test

{'salesfacts': {'tests': [],
  'cat_types': ['object', 'category'],
  'types': {'index': 'int',
   'ProductID': 'int',
   'Date': 'datetime',
   'Zip': 'int',
   'Units': 'int',
   'Revenue': 'float'},
  'max_null': 0.2},
 'products': {'tests': [],
  'cat_types': ['object', 'category'],
  'max_null': 0.2},
 'geo': {'tests': [], 'cat_types': ['object', 'category'], 'max_null': 0.2},
 'date': {'tests': [], 'cat_types': ['object', 'category'], 'max_null': 0.2},
 'manufacturer': {'tests': [],
  'cat_types': ['object', 'category'],
  'max_null': 0.2}}

## Processo completo

In [11]:
#Importando bibliotecas nescessárias 
import pandas as pd 
import json
import psycopg2
from sqlalchemy import create_engine
from test_functions import *

def conver_json(data):
    if type(data) != dict and type(data) != list:
        return str(data)
    else:
        if type(data) == dict:
            for key in data:
                data[key] = conver_json(data[key])
        elif type(data) == list:
            for cont in range(len(data)):
                data[cont] = conver_json(data[cont])
    return data
                

#Lendo configurações dos dados
config_file = open('../config/data_config.json')
config_data = json.loads(config_file.read())
config_file.close()

#Lendo configurações de test
test_file = open('../config/test_config.json')
config_test = json.loads(test_file.read())
test_file.close()

#Criando string de conexão
connect_data = config_data['connection']
textEngine = f"{connect_data['driver']}://{connect_data['user']}:{connect_data['password']}@{connect_data['host']}:{connect_data['port']}/{connect_data['database']}"
        
#lendo tabelas
out_data = {} #dicionario de testes das tabelas
for table_name in config_data['tables']:
    print(f"Tabela {table_name}:")
    df = pd.read_sql_table(table_name,textEngine)    
    list_functions = [test_columns,test_types,test_null_percent,test_n_uniques,test_category_percent]
    test_table = {} 
    for function in list_functions:
        name_test = function.__name__
        retorno = function(df,config_test[table_name])
        test_table[name_test] = retorno
    out_data[table_name] = test_table


try:
    with open('../test_out.json', 'w') as outfile:
        json.dump(out_data, outfile, indent=2)
        print("normal")
except TypeError as error:
    with open('../test_out.json', 'w') as outfile:
        json.dump(conver_json(out_data), outfile, indent=2)
        print('convert_json')
except Exception as e:
    print(e)

Tabela salesfacts:
Tabela products:
Tabela geo:
Tabela date:
Tabela manufacturer:
convert_json


In [32]:
dados = open('test_out.json').read()
out_json = json.loads(dados)

In [34]:
out_json.keys()

dict_keys(['salesfacts', 'products', 'geo', 'date', 'manufacturer'])